In [3]:
import numpy
import toyplot.text

text = "text"
style = {
    "font-size":"12px",
    "-toyplot-anchor-shift":"0",
    "text-anchor":"middle",
    "alignment-baseline":"middle",
    "baseline-shift":"0%",
}

left, right, top, bottom = toyplot.text.extents(text, style)
print left, right, top, bottom

[-24.] [ 24.] [-6.] [ 6.]
